In [4]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [12]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import mean
from pyspark.sql.functions import round, col

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [13]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

homes_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")


In [14]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('homes_data')
homes_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [16]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
fourbedhouse_avgprice = homes_df.filter(homes_df.bedrooms == 4)\
    .groupBy("date")\
    .agg(round(mean("price"), 2).alias("Average Price"))

fourbedhouse_avgprice.show()

+----------+-------------+
|      date|Average Price|
+----------+-------------+
|2021-11-13|     358445.0|
|2019-05-08|     234979.5|
|2021-08-27|    275547.25|
|2021-01-27|    367520.57|
|2021-06-22|     297708.8|
|2021-10-11|    283388.33|
|2020-08-24|    301080.75|
|2022-03-28|    338544.78|
|2019-06-04|    306373.08|
|2021-12-18|     305264.3|
|2020-11-29|    318153.89|
|2020-01-21|     420576.0|
|2020-07-24|     293661.5|
|2021-11-25|    291762.27|
|2019-11-18|    290259.83|
|2019-09-22|     284864.5|
|2020-08-05|    243835.56|
|2019-11-01|    254751.38|
|2021-10-02|    303838.38|
|2020-08-29|     285457.3|
+----------+-------------+
only showing top 20 rows



In [17]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
threebed_threebath_avgprice = homes_df.filter((homes_df.bedrooms == 3) & (homes_df.bathrooms == 3))\
    .groupBy("date")\
    .agg(round(mean("price"), 2).alias("Average Price"))

threebed_threebath_avgprice.show()

+----------+-------------+
|      date|Average Price|
+----------+-------------+
|2019-05-08|    285754.71|
|2021-06-22|     306565.0|
|2021-01-27|    385711.75|
|2020-08-24|    319252.33|
|2021-11-13|    348180.33|
|2022-03-28|     280464.4|
|2021-10-11|     289886.4|
|2021-12-18|     178508.5|
|2021-08-27|    298209.33|
|2019-06-04|     223109.0|
|2020-08-05|    304566.75|
|2019-11-18|    319833.67|
|2020-01-21|     303045.8|
|2021-11-25|    321816.67|
|2019-11-01|     279354.0|
|2020-07-24|     283740.0|
|2020-11-29|     366673.0|
|2021-10-02|     400307.5|
|2019-09-22|     350352.5|
|2020-04-30|     192065.5|
+----------+-------------+
only showing top 20 rows



In [18]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

threebed_threebath_twofloors_avgprice = homes_df.filter(
    (homes_df.bedrooms == 3) &
    (homes_df.bathrooms == 3) &
    (homes_df.floors == 2) &
    (homes_df.sqft_living >= 2000)
)\
    .groupBy("date")\
    .agg(round(mean("price"), 2).alias("Average Price"))

threebed_threebath_twofloors_avgprice.show()

+----------+-------------+
|      date|Average Price|
+----------+-------------+
|2021-06-22|     420422.0|
|2021-01-27|     446538.0|
|2021-12-18|     178508.5|
|2021-11-13|     697736.0|
|2021-08-27|     385686.0|
|2022-03-28|     204818.0|
|2021-10-11|     167892.0|
|2019-05-08|     391556.0|
|2019-11-01|    201940.33|
|2019-11-18|     398256.0|
|2021-10-02|     402262.0|
|2019-09-22|     365867.0|
|2020-01-21|     293205.0|
|2020-04-30|     232217.0|
|2022-01-31|     284811.0|
|2022-05-19|    302160.33|
|2020-08-29|     423501.0|
|2020-08-28|     296028.5|
|2019-11-21|     428167.0|
|2021-07-30|     346839.0|
+----------+-------------+
only showing top 20 rows



In [21]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

homeperview_avgprice = homes_df.filter(homes_df.price >= 350000) \
    .groupBy("view") \
    .agg(mean("price").alias("Average Price")) \
    .orderBy("view", ascending=False)

homeperview_avgprice .show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|     Average Price|
+----+------------------+
| 100|         1026669.5|
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 0.7963912487030029 seconds ---


In [22]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("homes_data")

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes_data')

True

In [26]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Assuming 'homes_df' is already cached
cached_data_reult = homes_df.filter(homes_df.price >= 350000) \
    .groupBy("view") \
    .agg(mean("price").alias("average_price")) \
    .orderBy("view", ascending=False)

cached_data_reult.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|     average_price|
+----+------------------+
| 100|         1026669.5|
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 2.010643482208252 seconds ---


The run time was over 2x the time from the first try.

In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_df.write.partitionBy('date_built').parquet('parquet_sales', mode='overwrite')

In [29]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('parquet_sales')

In [30]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_sales')

In [31]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Assuming you've already loaded the Parquet file into a DataFrame named 'parquet_df'
parquet_results = parquet_df.filter(parquet_df.price >= 350000) \
    .groupBy("view") \
    .agg(mean("price").alias("average_price")) \
    .orderBy("view", ascending=False)

parquet_results.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     average_price|
+----+------------------+
| 100|         1026669.5|
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 1.2756288051605225 seconds ---


This was about half the time of the cached result, but still longer than the first try.

In [32]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("homes_data")

In [34]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_data"):
    print("home_sales DataFrame is cached.")
else:
    print("home_sales DataFrame is not cached.")


home_sales DataFrame is not cached.
